# RAG's other half, Retriever!
1. Retriever
2. QA ChatBot
3. Conversational ChatBot
---

In [1]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

# import streamlit as st
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_groq import ChatGroq
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Fetching Api keys that are stored in '.env'
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.environ["GROQ_API_KEY"]

---
Loading Embeddings from stored Vector DataBase

---

In [3]:
from langchain_chroma import Chroma
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
db3 = Chroma(persist_directory="../chroma_db copy",embedding_function=embeddings)

In [12]:
simple_retriever= db3.as_retriever(search_kwargs={'k': 5})

In [13]:
simple_retriever.invoke("Summarize 'Unit 10-topic 1'")
                      

[Document(metadata={'source': '../data/transcriptions.json', 'topic': 'Unit1.webm'}, page_content="important thing that you'll have to learn to balance as well okay so for this wrapping up unit one i have a an exercise please read the three articles from the previous slide i've posted this up in discussions and write a short two hundred and fifty to five hundred word reflection on all three like you know what you think about this and this is an important time to to to think about and how i can see you bring together multiple p sources of information but also think about to me what is the the overarching strategic concept of this course which is how do we manage change which is a one way how we manage people that's why we have all these systems and that's what this course is all about i wanna be clear this is not a marked assignment but it's a really important a first opportunity for me to see your analytical and writing skills and provide support and direction for that so yeah checkout

---
SelfQueryRetriever for retrieving docs based on metadata.

---

In [14]:
metadata_field_info = [
    AttributeInfo(
        name="topic",
        description="The topic of that talk which is captured in a transcription form. One of ['Unit 10 - Topic 1.webm','Unit 10 - Topic 2.webm','Unit 10 - Topic 3.webm','Unit 2 - Intro & Getting Started with OOL.webm','Unit 2.1 - Topic 1 - Defining Projects.webm','Unit 2.1 - Topic 2 - The Purpose of Projects.webm','Unit 2.1 - Topic 3 - Benefits of Project Management.webm','Unit 2.1 - Topic 4 - Project Lifecycles.webm','Unit 2.1 - Topic 5 - Project Processes & Knowledge.webm','Unit 2.1 - Topic 6 - Organizational Project Management.webm','Unit 2.1 - Topic 7 - The Triple Constraint.webm','Unit 2.2 - Ethics & Professional Conduct.webm','Unit 2.2 - The PMI Values.webm','Unit 2.2 - Topic 1 - Ethics & Professional Conduct.webm','Unit 2.2 - Topic 2 - The PMI Values.webm','Unit 2.3 - Topic 1 - Project Management Roles.webm','Unit 2.3 - Topic 2 - PM as Conductor.webm','Unit 2.3 - Topic 3 - PM Spheres of Influence.webm','Unit 2.3 - Topic 4 - PM Competencies.webm','Unit 3.1 - Topic 1 - Project Integration Management.webm','Unit 3.1 - Topic 2 - Develop Project Charter.webm','Unit 3.1 - Topic 3 - Develop PM Plan.webm','Unit 3.1 - Topic 4 - Direct and Manage Project Work.webm','Unit 3.1 - Topic 5 - Manage Project Knowledge.webm','Unit 3.1 - Topic 6 - Monitor and Control Project Work.webm','Unit 3.1 - Topic 7 - Perform Integration Change Control.webm','Unit 3.1 - Topic 8 - Closing Project or Phase.webm','Unit 3.2 - Topic 1 - Scope Management.webm','Unit 3.2 - Topic 2 - Schedule Managment.webm','Unit 3.2 - Topic 3 - Cost Management.webm','Unit 4 - Topic 1 - Project Quality Management.webm','Unit 4 - Topic 2 - Manage Quality.webm','Unit 4 - Topic 3 - Project Resource Management.webm','Unit 4 - Topic 4 - Manage Team.webm','Unit 4 - Topic 5 - Project Risk Management.webm','Unit 4 - Topic 6 - Identify Risks.webm','Unit 4 - Topic 7 - Project Communication Management.webm','Unit 4 - Topic 8 - Manage Communications.webm','Unit 4 - Topic 9 - Project Stakeholder Management.webm','Unit 5 - Topic 2 - Ethics.webm','Unit 5 - Topic 3 - Guiding Principles.webm','Unit 5- Topic 1 - Understanding SM.webm','Unit 6 - Topic 1 - SM Roles.webm','Unit 6 - Topic 2 - SM Definitions.webm','Unit 6 - Topic 3 - Service Relationship.webm','Unit 7 - 4 Dimensions of SM - Topic 3.webm','Unit 7 - 4 Dimesions of SM - Topic 1.webm','Unit 7 - 4 Dimesions of SM - Topic 4.webm','Unit 8 - Service Value - Topic 1.webm','Unit 8 - Service Value - Topic 2.webm','Unit 9 - Intro.webm','Unit 9 - Topic 1.webm','Unit 9 - Topic 2 - Part 1.webm','Unit 9 - Topic 2 - Part 2.webm','Unit 9 - Topic 3.webm','Unit1.webm','Week 1 Survey Results.webm']",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="This is the file name from which have extracted transcription text from.",
        type="string",
    ),
]
document_content_description = "contains transcription of the video topics of Information Technology Operation, MGMT8680 - Spring 2020 course. "

llm = ChatGroq(groq_api_key=groq_api_key,model_name="mixtral-8x7b-32768")

retriever = SelfQueryRetriever.from_llm(
    llm,
    db3,
    document_content_description,
    metadata_field_info,
    search_kwargs={"k": 15},
    
)

---
# 1: QA Bot retriever [no history chat context]
---

In [15]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions based on the provided context only.
    Please provide a good response based on the question and context
    <context>
    {context}
    </context>
    Questions:{input}

    """
)

document_chain = create_stuff_documents_chain(llm,prompt)
retriever_chain = create_retrieval_chain(retriever,document_chain)

In [16]:
prompt =input("Input your prompt here")

if prompt:
    response = retriever_chain.invoke({"input":prompt})
    print(response['answer'])
    print("-"*50,end="\n\n\n")
    for i, doc in enumerate(response["context"]):
        print(doc.metadata)
        print("---------------------------------------------------------")

Unit 10 Topic 1 focuses on organizational change and the human dimension of change. Change can be scary for most people, including those in an IT organization, as it often involves new things to learn, new risks, and the fear of failure. When leading change management projects and processes, it is important to understand and acknowledge the feelings and engage with those feelings of the people going through the cycle of change. The human aspect of change is crucial in the entire unit, and it is important to remember that there is no project or service that does not require people to be successful.

The cycle of change consists of denial, anger, depression, and acceptance. During the first two phases of denial and anger, information and communication are most likely to benefit the people being supported. During the depression phase, emotional support is important. It is also essential to recognize and accept that not everyone will be able to move along with every single change, and the 

---
# 2: Conversational ChatBot retriever [History aware context]
---

In [17]:
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [18]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

---

# Test the Bot!

---

In [24]:
def generate_response():
    final_sources=[]
    # topic_set=set()
    # for con in response["context"]:
    #     topic_set.add(con.metadata["topic"])

    topic_set = {con.metadata["topic"] for con in response["context"]}

    if len(topic_set)<=3:
        for topic in topic_set: 
            final_sources.append(topic)
    else:
        final_sources = list(topic_set[:3])

    print(response['answer'])
    print(final_sources)

In [25]:
#First Set of Conversation

#session_id for maintaining the Chat's history during a session
response=conversational_rag_chain.invoke(
    {"input": "Summarize 'Unit 10 topic-1.webm'"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)
generate_response()

Unit 10, Topic 1, discusses managing organizational change, focusing on the integration of project management and service management practices, as well as considering the human factors of change. Change can provoke fear due to new things to learn, new risks, and the fear of failure. Effective change management involves understanding the human aspect of change, providing emotional support, acknowledging feelings, ensuring senior leadership support, and addressing negative attitudes. The goal is to bring as many people as possible along with the change, fostering a supportive environment and maintaining high morale.
['Unit 10 - Topic 1.webm']


In [26]:
response=conversational_rag_chain.invoke(
    {"input": "Shorten it"},
    config={"configurable": {"session_id": "abc123"}
    },
)
generate_response()

Unit 10, Topic 1, covers managing organizational change by integrating project and service management practices, while considering human factors. Fear of change arises from new things to learn, risks, and failure. Effective change management includes understanding the human aspect, providing emotional support, addressing negative attitudes, and ensuring senior leadership support to bring as many people as possible along with the change.
['Unit 10 - Topic 1.webm']


In [28]:
## For single messages
# session_id -> single QA[0|User_message & 1|AI_message] -> content(string)
store['abc123'].messages[1].content

"In Unit 10, the focus is on managing organizational change, specifically the integration of project management and service management practices while considering the human factors. Change can be scary due to new things to learn, new risks, and the fear of failure. When leading change management projects, it's important to understand the human dimension of change, such as providing emotional support and acknowledging feelings. Successful change brings along as many people as possible, addressing negative attitudes, and ensuring senior leadership support for a successful change project."